# CEDS ensemble of ERFaci

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import json
import scipy.stats
import pooch
import os
import glob
from tqdm.autonotebook import tqdm
from scipy.optimize import curve_fit

In [ ]:
files = glob.glob('../data/smith2021aerosol/*.csv')

ari = {}
aci = {}
models = []
years = {}
for file in files:
    model = os.path.split(file)[1][:-4]
    if model=='sumlog':
        continue
    models.append(model)
    df = pd.read_csv(file, index_col='year')
    ari[model] = (df['aprp_ERFariSW'] + df['aprp_ERFariLW']).values.squeeze()
    aci[model] = (df['aprp_ERFaciSW'] + df['aprp_ERFaciLW']).values.squeeze()
    years[model] = df.index
#pd.read_csv('../data/smith2021aerosol/')

In [ ]:
int(np.floor(years['E3SM'][0]-1850))

## Calibrate all models based on CMIP6 emissions time series

In [ ]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)

In [ ]:
emis_df = pd.read_csv(rcmip_emissions_file)

In [ ]:
bc = emis_df.loc[(emis_df['Scenario']=='ssp245')&
                 (emis_df['Region']=='World')&
                 (emis_df['Variable']=='Emissions|BC'), '1750':'2100'].interpolate(axis=1).squeeze().values

oc = emis_df.loc[(emis_df['Scenario']=='ssp245')&
                 (emis_df['Region']=='World')&
                 (emis_df['Variable']=='Emissions|OC'), '1750':'2100'].interpolate(axis=1).squeeze().values

so2 = emis_df.loc[(emis_df['Scenario']=='ssp245')&
                 (emis_df['Region']=='World')&
                 (emis_df['Variable']=='Emissions|Sulfur'), '1750':'2100'].interpolate(axis=1).squeeze().values

In [ ]:
aci['UKESM1-0-LL']

In [ ]:
def aci_log(x, beta, n0, n1, n2):
    aci = beta*np.log(1 + x[0]*n0 + x[1]*n1 + x[2]*n2)# + beta1*np.log(1 + x[1]/n1) + beta2*np.log(1 + x[2]/n2)
    aci_1850 = beta*np.log(1 + so2[100]*n0 + bc[100]*n1 + oc[100]*n2)
    return (aci - aci_1850)

In [ ]:
param_fits = {}

for model in models:
    ist = int(np.floor(years[model][0]-1750))
    ien = int(np.ceil(years[model][-1]-1750))
    print(model)
    param_fits[model], cov = curve_fit(
        aci_log, 
        [so2[ist:ien], bc[ist:ien], oc[ist:ien]],
        aci[model],
        bounds = ((-np.inf, 0, 0, 0), (0, np.inf, np.inf, np.inf)),
        max_nfev = 10000
    )

In [ ]:
param_fits[model]

In [ ]:
def aci_log1750(x, beta, n0, n1, n2):
    aci = beta*np.log(1 + x[0]*n0 + x[1]*n1 + x[2]*n2)
    aci_1750 = beta*np.log(1 + so2[0]*n0 + bc[0]*n1 + oc[0]*n2)
    return (aci - aci_1750)

In [ ]:
df_ar6 = pd.read_csv('../data/forcing/table_A3.3_historical_ERF_1750-2019_best_estimate.csv')

params_ar6, cov = curve_fit(
    aci_log1750, 
    [so2[:270], bc[:270], oc[:270]],
    df_ar6['aerosol-cloud_interactions'].values,
    bounds = ((-np.inf, 0, 0, 0), (0, np.inf, np.inf, np.inf)),
    max_nfev = 10000
)

In [ ]:
params_ar6

In [ ]:
fig, ax = pl.subplots(3, 4, figsize=(16, 12))
for imodel, model in enumerate(models):
    ax[imodel//4, imodel%4].plot(years[model], aci[model])
    ax[imodel//4, imodel%4].plot(np.arange(1750.5, 2101), aci_log([so2, bc, oc], *param_fits[model]))
    ax[imodel//4, imodel%4].set_title(model)
ax[2,3].plot(np.arange(1750.5, 2020), df_ar6['aerosol-cloud_interactions'].values)
ax[2,3].plot(np.arange(1750.5, 2101), aci_log1750([so2, bc, oc], *params_ar6))

In [ ]:
df_params = pd.DataFrame(param_fits, index=['beta', 'n0', 'n1', 'n2']).T

In [ ]:
fig, ax = pl.subplots(3, 4, figsize=(16, 12))
for imodel, model in enumerate(models):
    ax[imodel//4, imodel%4].plot(np.arange(1750.5, 2101), aci_log([so2, bc, oc], *param_fits[model]))
    ax[imodel//4, imodel%4].plot(np.arange(1750.5, 2101), aci_log1750([so2, bc, oc], *param_fits[model]))

In [ ]:
fig, ax = pl.subplots(3, 4, figsize=(16, 12))
for imodel, model in enumerate(models):
    ax[imodel//4, imodel%4].plot(np.arange(1750.5, 2101), aci_log([so2, bc, oc], *param_fits[model]) - aci_log1750([so2, bc, oc], *param_fits[model]))

In [ ]:
#df_params.to_csv('../data/smith2021aerosol/sumlog.csv')

In [ ]:
samples = 1500000

In [ ]:
df_params

In [ ]:
df_params.corr()

In [ ]:
beta_samp = df_params['beta']
n0_samp = (df_params['n0'])
n1_samp = (df_params['n1'])
n2_samp = (df_params['n2'])

In [ ]:
kde = scipy.stats.gaussian_kde([n0_samp, n1_samp, n2_samp])
aci_sample=kde.resample(size=samples*4, seed=63648708)

In [ ]:
aci_sample[1,:]
#np.exp(-436.18073312)

In [ ]:
aci_sample[0, aci_sample[0,:] < 0] = np.nan
aci_sample[1, aci_sample[1,:] < 0] = np.nan
aci_sample[2, aci_sample[2,:] < 0] = np.nan

In [ ]:
mask = np.any(np.isnan(aci_sample), axis=0)
aci_sample = aci_sample[:, ~mask]

In [ ]:
mask.sum()

In [ ]:
aci_sample

In [ ]:
NINETY_TO_ONESIGMA = scipy.stats.norm.ppf(0.95)
erfaci_sample = scipy.stats.uniform.rvs(size=samples, loc=-2.0, scale=2.0, random_state=71271)

## Use updated CEDS emissions

In [ ]:
emis_update_df = pd.read_csv('../data/emissions/co2_ch4_n2o_slcfs_1750-2020.csv', index_col=0)
so2 = emis_update_df.loc['Sulfur'].values
bc = emis_update_df.loc['BC'].values
oc = emis_update_df.loc['OC'].values

In [ ]:
beta = np.zeros(samples)
erfaci = np.zeros((271,samples))
for i in tqdm(range(samples)):
    ts2010 = np.mean(
        aci_log(
            [so2[255:265], bc[255:265], oc[255:265]], 
            0.92,
            aci_sample[0,i], aci_sample[1,i], aci_sample[2,i]
        )
    )
    ts1850 = aci_log(
        [so2[100], bc[100], oc[100]], 
        0.92,
        aci_sample[0,i], aci_sample[1,i], aci_sample[2,i]
    )
    ts1750 = aci_log(
        [so2[0], bc[0], oc[0]], 
        0.92,
        aci_sample[0,i], aci_sample[1,i], aci_sample[2,i]
    )
    erfaci[:,i] = (
        aci_log(
            [so2, bc, oc], 
            0.92,
            aci_sample[0,i], aci_sample[1,i], aci_sample[2,i]
        ) - ts1750
    )/(ts2010-ts1850)*(erfaci_sample[i])
    beta[i] = erfaci_sample[i] / (ts2010-ts1750)

In [ ]:
erfaci_sample[-1]

In [ ]:
pl.fill_between(np.arange(1750, 2021), np.nanpercentile(erfaci, 95, axis=1), np.nanpercentile(erfaci, 5, axis=1), color='lightgray');
pl.plot(np.arange(1750, 2021), np.nanpercentile(erfaci, 50, axis=1));

In [ ]:
pl.plot(erfaci[:, :20]);

In [ ]:
pl.plot(erfaci[100:, :20]/erfaci[100:, 0:1]);

In [ ]:
pl.hist(aci_sample[0,:])

In [ ]:
pl.hist(aci_sample[1,:])

In [ ]:
pl.hist(aci_sample[2,:])

In [ ]:
pl.hist(beta)

In [ ]:
pl.plot(erfaci[:, 7])
pl.plot(aci_log1750([so2, bc, oc], beta[7], *aci_sample[0:,7]))

In [ ]:
aci_log1750([so2, bc, oc], beta[0], *aci_sample[:,0])

In [ ]:
aci_sample.shape

In [ ]:
df = pd.DataFrame({
    'shape_so2' : aci_sample[0,:samples],
    'shape_bc' : aci_sample[1,:samples], 
    'shape_oc' : aci_sample[2,:samples], 
    'beta' : beta
})

In [ ]:
df.to_csv('../data/parameter_sets/erfaci_ceds2019.csv', index=False)